In [ ]:
import pandas as pd
data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/average GC full dataset new for colab 1007.csv')
!pip install xgboost==1.6.1
import pickle
from matplotlib import pyplot

In [ ]:
from sklearn.model_selection import train_test_split
X=data.iloc[:,:5].values
Y = data.iloc[:,5:].values

train_data,test_data,train_targets,test_targets=train_test_split(X,Y,test_size=0.11,random_state=42)
print(len(train_data))
print(len(test_data))
print(test_data[0])

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error
import numpy as np

def mae(actual,pred):
     return (mean_absolute_error(actual,pred))
from sklearn.metrics import mean_absolute_percentage_error
def mape(actual,pred):
    calc=np.mean(np.absolute((actual-pred)*100/actual))
    return calc

In [ ]:
from xgboost import XGBRegressor
xgbr=XGBRegressor()

taking default params

In [ ]:
defparam=xgbr.get_params()
print(defparam)

In [ ]:
xgbr.fit(train_data,train_targets)
predxgb=xgbr.predict(test_data)
print(predxgb)
print(test_targets)
print("mae:",mae(test_targets,predxgb))
print("mape:",mape(test_targets,predxgb))
#xgbr.save_model('/content/drive/MyDrive/Colab Notebooks/xgbr_avggc5in.h5')

individual parameter settings: mixed, but min child weight is altered

In [ ]:
xgbrne=XGBRegressor(n_estimators=20000,max_depth=8,min_child_weight=1,subsample=0.1,colsample_bytree=1,learning_rate=0.05)
xgbrne.fit(train_data,train_targets)
predxgbne=xgbrne.predict(test_data)
print(predxgbne)
print(test_targets)
print("mae:",mae(test_targets,predxgbne))
print("mape:",mape(test_targets,predxgbne))

random search on individual variables


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 200000, num = 50)]
# Create the random grid
random_grid = {'n_estimators': n_estimators}
xgbr_random = GridSearchCV(estimator = xgbr, param_grid= random_grid,  cv = 2, verbose=2, n_jobs = -1)
#xgbr_random = GridSearchCV(estimator = xgbr, param_distributions = random_grid, n_iter =100, cv = 3, verbose=0, random_state=42, n_jobs = -1)
# Fit the random search model
xgbr_random.fit(train_data, train_targets)

randomsearchcv


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 20000, num = 20)]
# min child weight
min_child_weight = [2,4,6,8,10]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# learning rate
learning_rate = [0.01,0.1,0.5,1]
# solsample by tree
colsample_bytree = [0.5,0.7,0.9]
#subsample
subsample=[0.5,0.7,0.9]

# Create the random grid
random_grid = {'max_depth': max_depth,
               'min_child_weight': min_child_weight,
               'learning_rate': learning_rate,
               'colsample_bytree': colsample_bytree,
               'n_estimators': n_estimators,
               'subsample': subsample,
              }
print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
xgbr_random = RandomizedSearchCV(estimator = xgbr, param_distributions = random_grid, n_iter =800, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
xgbr_random.fit(train_data, train_targets)

In [ ]:
print(xgbr_random.best_params_)

bayesian optimization method on certain features


In [ ]:
!pip install bayesian-optimization
from bayes_opt import BayesianOptimization

In [ ]:

#Xtra Gradient Boosting Machine
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
def xgbm_cl_bo(max_depth,min_child_weight, learning_rate, colsample_bytree,n_estimators, subsample):
    params_gbm = {}
    params_gbm['max_depth'] = round(max_depth)
    params_gbm['min_child_weight'] = min_child_weight
    params_gbm['learning_rate'] = learning_rate
    params_gbm['colsample_bytree'] = colsample_bytree
    params_gbm['n_estimators'] = round(n_estimators)
    params_gbm['subsample'] = subsample
    scores = cross_val_score(XGBRegressor(random_state=0, **params_gbm),
                             train_data,train_targets, cv=5).mean()
    score = scores.mean()
    return score

In [ ]:
# Run Bayesian Optimization
import time
start = time.time()
params_gbm ={
    'max_depth':(3, 10),
    'min_child_weight':(1,10),
    'learning_rate':(0.01, 1),
    'colsample_bytree':(0.5,0.9),
    'n_estimators':(80,5000),
    'subsample': (0.5, 0.9)
            }
gbm_bo = BayesianOptimization(xgbm_cl_bo, params_gbm, random_state=0)
gbm_bo.maximize(init_points=1000, n_iter=6)
print('It takes %s minutes' % ((time.time() - start)/60))

In [ ]:
params_gbm = gbm_bo.max['params']
params_gbm['max_depth'] = round(params_gbm['max_depth'])
params_gbm['n_estimators'] = round(params_gbm['n_estimators'])
params_gbm

{'colsample_bytree': 0.9,
 'learning_rate': 0.033727271369527585,
 'max_depth': 7,
 'min_child_weight': 1.8518249906475186,
 'n_estimators': 93,
 'subsample': 0.9}
 10/28
 mae: 2.8657398451131537
mape: 20.80642400363375

In [ ]:
xgbrnew=XGBRegressor(n_estimators=93,max_depth=7,min_child_weight=1.852,subsample=0.9,learning_rate=0.033727,colsample_bytree=0.9)
xgbrnew.fit(train_data,train_targets)
predxgb2=xgbrnew.predict(test_data)
print(predxgb2)
print(test_targets)
print("mae:",mae(test_targets,predxgb2))
print("mape:",mape(test_targets,predxgb2))